In [13]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import torch
import copy
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import Tensor

train = pd.read_csv('dataAll.csv')
test = pd.read_csv('testAll.csv')
train_data=train.iloc[:,:30].applymap(lambda x: x+126.23)
train_loc = train.iloc[:,30:]

x_test = test.applymap(lambda x: x+126.23)
x_test = torch.from_numpy(
       x_test.values.astype(np.float32))

train_feat = torch.from_numpy(
        train_data.values.astype(np.float32))
train_labels = torch.from_numpy(
        train_loc.values.astype(np.float32))

num_train = int(0.8 * train_feat.shape[0]) # 划分训练样本和验证集样本
indices = np.arange( train_feat.shape[0])
np.random.shuffle(indices)  # shuffle 顺序
train_indices = indices[:num_train]
valid_indices = indices[num_train:]

train_features = train_feat[train_indices,:]
train_label = train_labels[train_indices,:]
valid_features  = train_feat[valid_indices,:]
valid_labels = train_labels[valid_indices,:]

feat_dim = train_data.shape[0]

In [14]:
train_x_cord = train_label[:,0]
train_y_cord = train_label[:,1]
valid_x_cord = valid_labels[:,0]
valid_y_cord = valid_labels[:,1]

In [15]:
def error(y_test,y_pre):
    y_test = np.array(y_test)
    y_pre = np.array(y_pre)
    m = y_test-y_pre
    m=np.square(m)
    m=m.sum(axis=1)
    m=np.sqrt(m)
    return m.sum()/len(y_test)

def get_model(feat_dim,output_dim = 2 ):
    net = nn.Sequential(
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        
        nn.Linear(int(feat_dim), int(feat_dim*4/5)),
        nn.BatchNorm1d(int(feat_dim*4/5)),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        
        nn.Linear(int(feat_dim*4/5),int(feat_dim*3/5)),
        nn.BatchNorm1d(int(feat_dim*3/5)),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        
        nn.Linear(int(feat_dim*3/5),int(feat_dim*2/5)),
        nn.BatchNorm1d(int(feat_dim*2/5)),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        
        nn.Linear(int(feat_dim*2/5),int(feat_dim/5)),
        nn.BatchNorm1d(int(feat_dim/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim/5), output_dim)
        )
    return net

def get_data(x,y,batch_size,shuffle):
    dataset = TensorDataset(x,y)
    return DataLoader(dataset, batch_size, shuffle=shuffle,num_workers=4)

def train_model(model,x_train,y_train,x_valid,y_valid,epochs,batch_size,lr,weight_decay,use_gpu):
    
    if use_gpu:
        model = model.cuda()
    train_data = get_data(x_train,y_train,batch_size,True)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    #criterion = nn.CrossEntropyLoss()
    best_error = 0
    for e in  range(epochs):
        model.train()
        for data in train_data:
            x,y = data
            if use_gpu:
                x = x.cuda()
                y = y.cuda()
                
            out = model(x)
            loss = criterion(out, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if use_gpu:
                x_valid = x_valid.cuda()
        with torch.no_grad():
            output = model(x_valid)
        output = Tensor.cpu(output)
        out = output.numpy()
        print('epoch:{}'.format(e))
        error_i = error(out,y_valid)
        print('error:{}'.format(error_i)) 
        if e == 0:
            best_error = error_i
        elif error_i < best_error:
            best_model = copy.deepcopy(model)
    torch.save(best_model,"model_NN_master_baogao.t7")

def predict(x_test,use_gpu):
    net = torch.load('./model_NN_master_baogao.t7')
    if use_gpu:
        x_test = x_test.cuda()
    with torch.no_grad():
        a = Tensor.cpu(net(x_test)).numpy()
    #a = net(x_test).detach().numpy()
    #a = np.argmax(a,axis=1)
    #np.savetxt('NN_ans.txt',np.array(a),fmt='%f')
    #print(a)
    a = pd.DataFrame(a)
    a.to_csv('NN_base_baogao.csv',header=None)
    '''
    with open('./NN_master_ans.txt','w') as f:
        for i in a:
            f.write('{}'.format(i))
            f.write('\n')'''
    

In [16]:
# 可以调整的超参
batch_size = 128
epochs = 40000
lr = 0.001
wd = 0
use_gpu = False
feat_dim = train_feat.shape[1]
net = get_model(feat_dim)
train_model(net, train_features, train_label, valid_features, valid_labels, epochs, 
            batch_size, lr, wd, use_gpu)
predict(x_test,use_gpu)

epoch:0
error:37.96455318159767
epoch:1
error:37.83386056759379
epoch:2
error:37.698786687419144
epoch:3
error:37.56864236335705
epoch:4
error:37.40670480271669
epoch:5
error:37.255012936610605
epoch:6
error:37.07377405401035
epoch:7
error:36.89003628315007
epoch:8
error:36.67956621927555
epoch:9
error:36.45621513179172
epoch:10
error:36.214323253557566
epoch:11
error:35.964330833602844
epoch:12
error:35.691623544631305
epoch:13
error:35.42465434993532
epoch:14
error:35.11154794631307
epoch:15
error:34.79392888906857
epoch:16
error:34.46856807891332
epoch:17
error:34.11313722913972
epoch:18
error:33.68530684023286
epoch:19
error:33.26946050291074
epoch:20
error:32.8273973156533
epoch:21
error:32.32887289780078
epoch:22
error:31.793522093305302
epoch:23
error:31.349687702134542
epoch:24
error:30.83712504042691
epoch:25
error:30.29467678686934
epoch:26
error:29.74506791720569
epoch:27
error:29.193902611578267
epoch:28
error:28.690243976390686
epoch:29
error:28.141693786384216
epoch:30
er

epoch:240
error:10.389481044833442
epoch:241
error:10.392066471943727
epoch:242
error:10.063625005053364
epoch:243
error:10.614687348399094
epoch:244
error:10.430606580489973
epoch:245
error:10.131412718305304
epoch:246
error:10.511809710543337
epoch:247
error:10.08573283877749
epoch:248
error:10.02453597489489
epoch:249
error:10.047984844962807
epoch:250
error:10.528088489448576
epoch:251
error:10.217868819736417
epoch:252
error:10.338920247816947
epoch:253
error:10.119195580328267
epoch:254
error:10.479242677676261
epoch:255
error:10.220551524094438
epoch:256
error:10.466679384298189
epoch:257
error:10.434095296329236
epoch:258
error:10.144523038284282
epoch:259
error:10.494745133610932
epoch:260
error:10.045442371442432
epoch:261
error:10.371566239489004
epoch:262
error:10.255724829196312
epoch:263
error:10.436942866672057
epoch:264
error:10.185698475905562
epoch:265
error:10.309526095569211
epoch:266
error:10.028373372816947
epoch:267
error:10.323602239650711
epoch:268
error:10.386

epoch:477
error:10.252596797178201
epoch:478
error:10.136263947283311
epoch:479
error:10.18762128072445
epoch:480
error:10.271515958521992
epoch:481
error:10.516383636198254
epoch:482
error:10.328637916397154
epoch:483
error:10.005486689440492
epoch:484
error:10.321426766655886
epoch:485
error:10.1095367076326
epoch:486
error:10.132470766292045
epoch:487
error:10.159065986820828
epoch:488
error:10.170364044307892
epoch:489
error:10.102196697121604
epoch:490
error:9.980816168741915
epoch:491
error:10.160205520294308
epoch:492
error:10.225515190410738
epoch:493
error:10.160840980756792
epoch:494
error:10.2259933649337
epoch:495
error:10.243561383206663
epoch:496
error:10.092313581419793
epoch:497
error:9.967226410899094
epoch:498
error:10.256043822768435
epoch:499
error:10.267716460826326
epoch:500
error:10.218329307487064
epoch:501
error:10.193614569857697
epoch:502
error:10.261623367763583
epoch:503
error:10.235836053727361
epoch:504
error:9.974873413243856
epoch:505
error:10.075809296

epoch:714
error:9.940716465879689
epoch:715
error:10.240134571070504
epoch:716
error:10.185322000323415
epoch:717
error:10.039516039375808
epoch:718
error:10.197476602926908
epoch:719
error:10.062838575355757
epoch:720
error:10.269497771466689
epoch:721
error:10.082083046976066
epoch:722
error:10.142354513664294
epoch:723
error:10.157731267181436
epoch:724
error:10.190408210705046
epoch:725
error:10.032208244057244
epoch:726
error:10.46053196757762
epoch:727
error:10.279338565249029
epoch:728
error:10.194999191461838
epoch:729
error:10.014047087241268
epoch:730
error:10.170583233950518
epoch:731
error:10.054394404915913
epoch:732
error:10.100656684589262
epoch:733
error:9.976741894404915
epoch:734
error:10.025812580853817
epoch:735
error:10.339663723924645
epoch:736
error:10.043841718547865
epoch:737
error:10.0681168135511
epoch:738
error:9.983707956015524
epoch:739
error:9.901396244340233
epoch:740
error:10.088000535656533
epoch:741
error:9.954394657584087
epoch:742
error:9.9583981848

epoch:953
error:10.202988559184993
epoch:954
error:10.056089808376456
epoch:955
error:9.959087968952135
epoch:956
error:9.956247978654593
epoch:957
error:10.073434215313712
epoch:958
error:10.064583880781047
epoch:959
error:9.91496010369502
epoch:960
error:9.922475086917853
epoch:961
error:9.924054263017464
epoch:962
error:10.245935832390039
epoch:963
error:10.192817401762614
epoch:964
error:9.875500282988357
epoch:965
error:9.783378729382276
epoch:966
error:9.950127092092497
epoch:967
error:9.933053040103493
epoch:968
error:10.134828792043985
epoch:969
error:9.931564824547218
epoch:970
error:10.091502516575032
epoch:971
error:10.020255144324063
epoch:972
error:10.192002546895214
epoch:973
error:9.993913223641655
epoch:974
error:9.954654905805304
epoch:975
error:10.117361841041397
epoch:976
error:10.228804930061449
epoch:977
error:9.91328996705207
epoch:978
error:9.900475268838939
epoch:979
error:9.887344735405886
epoch:980
error:10.125801589788162
epoch:981
error:10.014017398730594
ep

epoch:1187
error:10.01045667246119
epoch:1188
error:10.032494390766495
epoch:1189
error:9.956480433376456
epoch:1190
error:9.98771464161546
epoch:1191
error:10.07779653137128
epoch:1192
error:10.039722595609637
epoch:1193
error:10.012359263826003
epoch:1194
error:10.01184192573577
epoch:1195
error:10.016251617076326
epoch:1196
error:9.904448475905562
epoch:1197
error:10.010549528015847
epoch:1198
error:9.879523392019728
epoch:1199
error:9.869351602926908
epoch:1200
error:9.836576750485122
epoch:1201
error:9.874284949062096
epoch:1202
error:10.298535029915913
epoch:1203
error:10.116081445059832
epoch:1204
error:9.88692088454075
epoch:1205
error:10.021225390119664
epoch:1206
error:9.914965788728978
epoch:1207
error:9.768822515766495
epoch:1208
error:9.983406649215718
epoch:1209
error:9.985366722590557
epoch:1210
error:10.199317290588615
epoch:1211
error:10.050524791801424
epoch:1212
error:9.911924927231565
epoch:1213
error:10.012147022558215
epoch:1214
error:10.21509389149418
epoch:1215


epoch:1420
error:10.28422516777167
epoch:1421
error:10.18636046652652
epoch:1422
error:9.852390619946636
epoch:1423
error:10.015959785333118
epoch:1424
error:9.894180041235446
epoch:1425
error:10.091499358222833
epoch:1426
error:9.969983652369017
epoch:1427
error:10.05732661909767
epoch:1428
error:10.117697889715394
epoch:1429
error:10.163089727522639
epoch:1430
error:10.055248423350582
epoch:1431
error:10.15902240156048
epoch:1432
error:10.224229741065653
epoch:1433
error:10.269346802231565
epoch:1434
error:9.96309528622251
epoch:1435
error:9.963182456743208
epoch:1436
error:10.0257980524337
epoch:1437
error:9.962610163324708
epoch:1438
error:9.96452222974612
epoch:1439
error:10.081552443806597
epoch:1440
error:10.027317219841526
epoch:1441
error:9.90942477563066
epoch:1442
error:9.708288274175292
epoch:1443
error:10.104555354543985
epoch:1444
error:10.097176812136158
epoch:1445
error:10.161222509702458
epoch:1446
error:9.929438621846701
epoch:1447
error:10.08935483707956
epoch:1448
e

epoch:1652
error:10.117485648447607
epoch:1653
error:9.945700977320504
epoch:1654
error:9.923479442917206
epoch:1655
error:10.094829524781694
epoch:1656
error:10.117982141413325
epoch:1657
error:10.159816411303364
epoch:1658
error:10.10567593790427
epoch:1659
error:9.831678777894567
epoch:1660
error:10.04040795803687
epoch:1661
error:9.931457440572444
epoch:1662
error:9.997184644849613
epoch:1663
error:9.711753618208279
epoch:1664
error:10.006016029269082
epoch:1665
error:10.025609182972186
epoch:1666
error:9.952671460624192
epoch:1667
error:10.021630922542045
epoch:1668
error:9.961778885025874
epoch:1669
error:9.798471231201487
epoch:1670
error:9.968165073172704
epoch:1671
error:9.958559892464425
epoch:1672
error:9.932599500727685
epoch:1673
error:9.845248322283311
epoch:1674
error:9.76233273366753
epoch:1675
error:9.789641110122897
epoch:1676
error:9.809747180223157
epoch:1677
error:9.966860673714425
epoch:1678
error:9.96764773508247
epoch:1679
error:10.279354988680465
epoch:1680
err

KeyboardInterrupt: 